## CS411 Homework 4
Alex Lewandowski

10/31/20

## 3.2/8) 

**Consider the problem of counting, in a given text, the number of substrings that start with an A and end with a B . For example, there are four such substrings in CABAAXBYA .**

**a. Design a brute-force algorithm for this problem and determine its efficiency class.**

* Initialize count = 0
* Iterate over the input string
    * If an 'A' is encountered, iterate over the remaining substring
        * If a 'B' is encountered, increment the count
* return the count

In [ ]:
def count_AB_substrings_brute_1(s: str) -> int:
    count = 0
    for i, char_1 in enumerate(s):
        if char_1 == 'A':
            for char_2 in s[i+1:]:
                if char_2 == 'B':
                    count += 1
    return count

In [ ]:
text = ["CABAAXBYA", "", "AAAAAA", "BBBBB", "ABABAB", "ABABABABABAXXAXXXXXAXXXXXXAXX"]

for word in text:
    print(count_AB_substrings_brute_1(word))

**b. Design a more efficient algorithm for this problem. [Gin04]**

A still brute force yet more efficient solution is to stop checking substrings when it is no longer possible to find the pattern (i.e when there are no more "B"s in the remaining substrings). This will save time on strings that look like "ABABABAXXAXXXXXAXXX...XXXAXX," where the substring indicated by the ellipses contains many "A"s and no "B"s.

* Initialize count = 0
* Intialize b_count = 0
* Iterate over the input string
    * If an 'A' is encountered, iterate over the remaining substring
        * If a 'B' is encountered, increment count and b_count
    * If b_count == 0, return count 
    * else b_count = 0
* return the count

In [ ]:
def count_AB_substrings_brute_2(s: str) -> int:
    count = 0
    b_count = 0
    for i, char_1 in enumerate(s):
        if char_1 == 'A':
            for char_2 in s[i+1:]:
                if char_2 == 'B':
                    count += 1
                    b_count += 1
            if b_count == 0:
                return count
        b_count = 0
    return count

In [ ]:
text = ["CABAAXBYA", "", "AAAAAA", "BBBBB", "ABABAB", "ABABABABABAXXAXXXXXAXXXXXXAXX"]

for word in text:
    print(count_AB_substrings_brute_2(word))

## 4.5/13) 
**You need to search for a given number in an n × n matrix in which every row and every column is sorted in increasing order. Can you design a O(n) algorithm for this problem? [Laa10]**

* Iterate over n
    * If matrix\[i\]\[0\] <= value <= matrix\[i\]\[n-1\]
        * Iterate over row i
            * If matrix\[i\]\[j\] == value
                * return \[i, j\]
* return \[\]

In [ ]:
import numpy as np

def find_num(matrix : np.ndarray, val : int) -> list:
    n = len(matrix)
    for i in range(0, n):
        if val >= matrix[i][0] and val <= matrix[i][n-1]:
            for j, num in enumerate(matrix[i]):
                if num == val:
                    return [i, j]
    return list()

In [ ]:
for i in range(0, 1000):
    fail_count = 0
    n = np.random.randint(3, 100)
    x = np.random.choice(10000, n**2, replace=False)
    x.sort()
    x = np.reshape(x, (n, n))
    row = np.random.randint(n-1)
    col = np.random.randint(n-1)
    indices = find_num(x, x[row][col])
    if x[row][col] != x[indices[0], indices[1]]:
        print("FAILED")
        print(f'{x}')
        print(f"[row][col]: [{row}][{col}]")
        print(f"indices: {indices}")
        fail_count += 1
if fail_count == 0:
    print("ALL TESTS PASSED")
else: 
    print(f"{fail_count} TESTS FAILED")

## 5.1/5)
**Find the order of growth for solutions of the following recurrences.**

**a. T (n) = 4T (n/2) + n, T (1) = 1**


a = 4

b = 2

d = 1

a > b<sup>d</sup>

4 > 2<sup>1</sup>

T(n)~θn<sup>log<sub>2</sub>4</sup>

**b. T (n) = 4T (n/2) + n<sup>2</sup> , T (1) = 1**

a = 4

b = 2

d = 2

a = b<sup>d</sup>

4 = 2<sup>2</sup>

T(n)~θn<sup>2</sup>log<sub>2</sub>n

**c. T (n) = 4T (n/2) + n<sup>3</sup> , T (1) = 1**

a = 4

b = 2

d = 3

a < b<sup>d</sup>

4 < 2<sup>3</sup>

T(n)~θn<sup>3</sup>

## 5.1/11

### OMG I did a problem from the wrong section. I had fun coding Trominos and I'm leaving it in because it makes me sad to delete it. 5.2/11 follows.

*Tromino puzzle* **A tromino (more accurately, a right tromino) is an L-shaped
tile formed by three 1 × 1 squares. The problem is to cover any 2<sup>n</sup> × 2<sup>n</sup> chessboard with a missing square with trominoes. Trominoes can be oriented in an arbitrary way, but they should cover all the squares of the board except the missing one exactly and with no overlaps. [Gol94]**

**Design a divide-and-conquer algorithm for this problem.**

A Tromino board can be described with an n value to determine its size, and a set of x, y values representing the location of the missing square.

A tromino can be represented with a set of 3 points on the x, y plane.

* call the tiling function on the empty tromino board
* if the n = 1
    * place a tromino in the only possible location
    * return
* divide the board into quartiles
* mark the 4 adjoining corner squares at the center of the current board as missing
* determine which quartile contains the passed in missing square
* unmark that quartile's corner square and mark the correct missing square
* recursively call the tiling function on each quartile
* place a tromino in the L-shaped spot created by the 3 quartiles with adjoining missing tiles

In [ ]:
import numpy as np

class Square:
    def __init__(self, row: int, col: int):
        self.row = row
        self.col = col
        
    def __eq__(self, other): 
        if not isinstance(other, Square):
            return NotImplemented
        return self.row == other.row and self.col == other.col

class Tromino:
    def __init__(self, sq_1=Square(-1, -1), sq_2=Square(-1, -1), sq_3=Square(-1, -1)):
        self.sq_1 = sq_1
        self.sq_2 = sq_2
        self.sq_3 = sq_3
    
class TrominoBoard:
    def __init__(self, n: int, missing_sq: Square):
        self.trominos = list()
        self.n = n
        self.missing_sq = missing_sq
        self.used_squares = np.zeros((2**n, 2**n), dtype=bool)
        self.used_squares[self.missing_sq.row][self.missing_sq.col] = True
        
    def id_partial_quartile(self, llc, urc, missing_sq):
        q_width = (urc.col - llc.col + 1)//2
        quartile = ""
        if missing_sq.row <= llc.row - q_width:
            quartile = "u"
        else:
            quartile = "l"
        if missing_sq.col <= urc.col - q_width:
            quartile = f"{quartile}lq"
        else:
            quartile = f"{quartile}rq"
        return quartile
    
    def divide_quartiles(self, llc, urc):
        q_width = (urc.col - llc.col + 1)//2
        return {"ulq": {'llc': Square(llc.row - q_width, llc.col),
                        'urc': Square(urc.row, urc.col - q_width), 
                        "missing": Square(llc.row - q_width, urc.col - q_width)},
                "urq": {'llc': Square(llc.row - q_width, llc.col + q_width), 
                        'urc': Square(urc.row, urc.col), 
                        "missing": Square(llc.row - q_width, llc.col + q_width)},
                "llq": {'llc': Square(llc.row, llc.col), 
                        'urc': Square(urc.row + q_width, urc.col - q_width), 
                        "missing": Square(urc.row + q_width, urc.col - q_width)},
                "lrq": {'llc': Square(llc.row, llc.col + q_width), 
                        'urc': Square(urc.row + q_width, urc.col), 
                        "missing": Square(urc.row + q_width, llc.col + q_width)}}
    
    def place_tromino(self, tromino):
        self.used_squares[tromino.sq_1.row][tromino.sq_1.col] = True
        self.used_squares[tromino.sq_2.row][tromino.sq_2.col] = True
        self.used_squares[tromino.sq_3.row][tromino.sq_3.col] = True
        self.trominos.append(tromino)          
    
    def tile_board(self, llc=None, urc=None, missing_sq=None):
        if not (missing_sq or llc or urc):
            print("Entered with initial empty Tromino board")
            missing_sq = self.missing_sq
            llc = Square((2**self.n)-1, 0)
            urc = Square(0, (2**self.n)-1)
        else:
            print(f"recursive call on sub-board having lower-left-corner: ({llc.row}, {llc.col}) and upper-right-corner: ({urc.row}, {urc.col})")
        n = int(np.log2(urc.col - llc.col + 1))
        # base case
        if n == 1:
            squares = [Square(llc.row, llc.col),
                      Square(llc.row, urc.col),
                      Square(urc.row, llc.col),
                      Square(urc.row, urc.col)]
            squares.remove(missing_sq)
            tromino = Tromino(squares[0], squares[1], squares[2])
            self.place_tromino(tromino)
            return
        quartiles = self.divide_quartiles(llc, urc)
        incomplete_quartile = self.id_partial_quartile(llc, urc, missing_sq)
        quartiles[incomplete_quartile]['missing'] = missing_sq

        # recursive calls
        self.tile_board(quartiles['ulq']['llc'], 
                        quartiles['ulq']['urc'], 
                        quartiles['ulq']['missing'])
        
        self.tile_board(quartiles['urq']['llc'], 
                        quartiles['urq']['urc'], 
                        quartiles['urq']['missing'])

        self.tile_board(quartiles['llq']['llc'], 
                        quartiles['llq']['urc'], 
                        quartiles['llq']['missing'])
        
        self.tile_board(quartiles['lrq']['llc'], 
                        quartiles['lrq']['urc'], 
                        quartiles['lrq']['missing'])
        
        squares = list()
        quartiles.pop(incomplete_quartile)
        for q in quartiles:
            squares.append(quartiles[q]['missing'])
        # place remaining tromino
        self.place_tromino(Tromino(squares[0], squares[1], squares[2]))


In [ ]:
missing = Square(5, 5)
board = TrominoBoard(3, missing)

print("\nEmpty Board (True == occupied square):")
print(f"{board.used_squares}\n")

board.tile_board()

print("\nPLACED TROMINOS:")
for i, t in enumerate(board.trominos):
    print(f" Tromino {i}: sq_1: ({t.sq_1.row}, {t.sq_1.col}), sq_2: ({t.sq_2.row}, {t.sq_2.col}), sq_3: ({t.sq_3.row}, {t.sq_3.col})")

print("\nAfter Tiling (True == occupied square):")
print(board.used_squares)

## 5.2/11
**Nuts and bolts You are given a collection of n bolts of different widths and
n corresponding nuts. You are allowed to try a nut and bolt together, from
which you can determine whether the nut is larger than the bolt, smaller than
the bolt, or matches the bolt exactly. However, there is no way to compare
two nuts together or two bolts together. The problem is to match each bolt
to its nut. Design an algorithm for this problem with average-case efficiency
in θ(nlogn). [Raw91**

![A walkthrough of the quicksort algorithm applied to 2 arrays](quicksort_2_arrays.jpg)

This can be solved with quicksort, which has average-case efficiency in θ(nlogn). 

* Select nuts or bolts as an array of pivots and the other as a sorting array
* Add a pivot to the end of the sorting array
* iterate over sorting array
    * values < pivot stay where they are
    * values > pivot move to end of array
* repeat until all pivots have been used
* each nut will sit next to its cooresponding bolt in the sorted array

## 5.3/5.
**In each part, give a list of the values in the order they will be visited.**

**Traverse the following binary tree**

![](5_3_5.png)

**a. in preorder.**

a, b, d, e, c, f

**b. in inorder.**

d, b, e, a, c, f

**c. in postorder.**

d, e, b, f, c, a

## 6.1/8.
**You have a list of n open intervals (a<sub>1</sub> , b<sub>1</sub> ), (a<sub>2</sub> , b<sub>2</sub> ), . . . , (a<sub>n</sub> , b<sub>n</sub> ) on the real line.
(An open interval (a, b) comprises all the points strictly between its endpoints
a and b, i.e., (a, b) = {x| a < x < b}.) Find the maximum number of these
intervals that have a common point. For example, for the intervals (1, 4),
(0, 3), (−1.5, 2), (3.6, 5), this maximum number is 3. Design an algorithm
for this problem with a better than quadratic time efficiency.**

* create a (2n, 2) matrix of [points, bools] (bools indicate left or right endpoint)
* Sort matrix on endpoint values (use heapsort: O(logn))
* initialize current and max counts = 0
* iterate through the list of endpoints (O(n))
    * if element is a left enfpoint, current++
    * else, current--
    * if max < current, max = current
* return max

n * logn ~ O(nlogn)

In [ ]:
import numpy as np

def max_common_point(x: np.ndarray) -> int:
    x = np.stack(x, axis=1)
    a = np.stack([x[0], np.zeros(len(x[0]), dtype=bool)], axis=1)
    b = np.stack([x[1], np.ones(len(x[1]), dtype=bool)], axis=1)
    x = np.concatenate([a, b])
    x = x[x[:,0].argsort(kind='heapsort')]
    print("Sorted List of Endpoints:")
    print("(0 = left, 1 = right)")
    print(f"{x}\n")
    cur = 0
    mx = 0
    for pt in x:
        if not pt[1]:
            cur += 1
        else:
            cur -= 1
        if mx < cur:
            mx = cur
    return mx

In [ ]:
x = np.array([[1, 4], [0, 3], [-1.5, 2], [3.6, 5]])

print(max_common_point(x))

## 6.6/6.
**Consider the problem of finding, for a given positive integer n, the pair of integers whose sum is n and whose product is as large as possible. Design an efficient algorithm for this problem and indicate its efficiency class.**

The maximum product for *a* and *b* values that sum to n will be achieved by having the highest possible values for *a* and *b*.

* Do integer division of n by 2 (giving *a*)
* b = n - a
* return a and b

O(1)

In [ ]:
def get_max_a_b(n: int) -> list:
    if n > 1:
        a = n // 2
        return [a, n-a]
    return None

In [ ]:
print(get_max_a_b(15))
print(get_max_a_b(8))

## 7.1/6.
**The ancestry problem asks to determine whether a vertex u is an ancestor
of vertex v in a given binary (or, more generally, rooted ordered) tree of n
vertices. Design a O(n) input-enhancement algorithm that provides sufficient
information to solve this problem for any pair of the tree’s vertices in constant
time.**

![](postorder_traversal_diagram.jpg)


* Perform a postorder traversal of the tree, strarting from vertex u, searching for vertex v
* If vertex v is found before vertex u, vertex u is an ancestor of vertex v

O(n)

## 7.1/7.
**In part b, describe an algorithm.**

**The following technique, known as virtual initialization, provides a time-efficient way to initialize just some elements of a given array A[0..n − 1] so that for each of its elements, we can say in constant time whether it has been initialized and, if it has been, with which value. This is done by utilizing a variable counter for the number of initialized elements in A and two auxiliary arrays of the same size, say B[0..n − 1] and C[0..n − 1], defined as follows. 
B[0], . . . , B[counter − 1] contain the indices of the elements of A that were
initialized: B[0] contains the index of the element initialized first, B[1] contains
the index of the element initialized second, etc. Furthermore, if A[i] was the
kth element (0 ≤ k ≤ counter − 1) to be initialized, C[i] contains k.**

**a. Sketch the state of arrays A[0..7], B[0..7], and C[0..7] after the three assignments**

**A[3] ← x;**

**A[7] ← z;**

**A[1] ← y.**

A = [  , y,  , x,  ,  ,  , z]
B = [ 3, 7, 1,  ,  ,  ,  ,  ]
C = [  , 2,  , 0,  ,  ,  , 1]

**b. In general, how can we check with this scheme whether A[i] has been
initialized and, if it has been, with which value?**

counter = 3

B[C[3]] = 3 

A[3] = x

B[C[7]] = 7    

A[7] = z

B[C[1]] = 1      

A[1] = y
    
If B[C[i]] = i, A[i] has been initialized.

If A[i] is initialized, it holds the intialized value.

I'm not sure I understand the point of the counter. You could use it to make sure a new value hadn't been assigned to a previously initialized location.  

## 7.3/3.
**Why is it not a good idea for a hash function to depend on just one letter (say,
the first one) of a natural-language word?**

If a hash function depends on a single letter, you will end up with many collisions, resulting in clustering in the hash table.